In [ ]:
import numpy as np
import scipy.linalg as splin
from scipy.integrate import LSODA, solve_ivp
from consts import *
from lyapunov import *
from potential import *
from utils import *

In [ ]:
# Initial conditions (alpha0) and Jacobi interval
alpha0 = 50/384400 # initial guess for alpha

# C = 2U(r) - (vx^2 + vy^2)
C_range = [3.1370, 3.1493]

In [ ]:
#def mainFun(XL: np.ndarray, mu: float, C_range, IG):
    # XL is the lagrangian point position vector
    # mu is the relative gravitational parameter
    # C_range is the range of Jacobi constants to search for
    # IG is the initial guess for alpha (for Lyapunov orbit)
    


# Compute A at XL
A_xl = A(XL1.flatten()[:2], mu)
# Diagonalize it
L, V = splin.eig(A_xl)

# Filter small values - eps = 1e-14
# V.real[np.abs(V.real) <= eps] = 0
# V.imag[np.abs(V.imag) <= eps] = 0

# Filter complex numbers with positive imaginary part
Sel =  [(i, x) for i, x in enumerate(L) if isinstance(x, complex) and x.imag > 0]
i_sel = Sel[0][0]
L_sel = Sel[0][1]

# Now take the eigenvector corresponding to that eigenvalue

V_sel = V[:, i_sel]
# Angular velocity and period
om_ly = L_sel.imag
T_ly = 2*np.pi/om_ly
# Lyapunov linearized state 
# x(t) = XL + alpha0(cos(om_ly*t)*Re(V_sel) - sin(om_ly*t)*Im(V_sel))



t_span = np.linspace(0, 2*np.pi/om_ly, 1000)

def f_lyap(XL, om_ly, alpha0, V_sel, points = 1000):
    # Preallocate x
    x = np.zeros((4, points))
    for i in range(points):
        x[:, i] = XL + alpha0*(np.cos(om_ly*t_span[i])*V_sel.real - np.sin(om_ly*t_span[i])*V_sel.imag)
    return x



X_lyap = f_lyap(XL1.flatten(), om_ly, alpha0, V_sel)

    

In [ ]:
# Plotting cell
import matplotlib.pyplot as plt


# Plot the Lagrange point
plt.plot(XL1[0], XL1[1], 'ro', label='L1')

# Set the aspect ratio to be equal
plt.axis('equal')


# From X_lyap take the solution that crosses the x-axis
i_cross = [idx for idx in range(len(X_lyap)-1) if X_lyap[1, idx] * X_lyap[1,idx+1] < 0 and X_lyap[1, idx] > 0]
print(i_cross)
# Plot the Lyapunov orbit
plt.plot(X_lyap[0,:], X_lyap[1,:], label='Lyapunov orbit')
# Plot the crossing point
plt.plot(X_lyap[0,i_cross], X_lyap[1,i_cross], 'ro', label='Crossing point')
print(X_lyap[:, i_cross[0]])

#Plot initial position
plt.plot( X_lyap[0,0], X_lyap[1,0], 'g.', label='Initial position')
plt.grid()


plt.legend()
plt.show()

# Compute the Jacobi constant
C_lyap = Jacobi(X_lyap[0:2,0], X_lyap[2:4,0], mu)
print(f'Jacobi constant: {C_lyap}')



In [ ]:
# Take one of the crossing points 
X0 = X_lyap[:, i_cross[0]]
for i in range(4):
    if abs(X0[i]) < eps:
        X0[i] = 0
# Concat the STM
PHI0 = np.eye(4, 4)
Y0 = np.concatenate((X0, PHI0.flatten()))
# Lets integrate the equations of motion
tau_span = np.linspace(0, 1/2, 100)

sol = solve_ivp(fdyn, [0, 1/2], Y0, args = (T_ly, mu), t_eval=tau_span, method='LSODA', rtol=3*eps, atol=eps)

# Plot the solution
plt.plot(sol.y[0, :], sol.y[1, :], 'b')
plt.plot(XL1[0], XL1[1], 'ro')


In [ ]:
# Build the initial guess for iteration
Y_guess = np.array([X0[0], X0[3], T_ly]).reshape(3,1)
Yt = [0,0]
F_X = 1


while np.linalg.norm(F_X) > 1e-12:
    # Shoot
    DX, DF, F_X = shooting(Y_guess, mu)
    # Update the guess
    Y_guess = Y_guess + DX
    



In [ ]:
# Now integrate the equations of motion
Y_guess = Y_guess.flatten()
Y0 = [Y_guess[0], 0, 0, Y_guess[1]]
Y0 = np.concatenate((Y0, PHI0.flatten()))
T = Y_guess[2]

tau_span = np.linspace(0, 1, 1000)
sol_nl = solve_ivp(fdyn, [0, 1], Y0, args = (T, mu), t_eval=tau_span, method='LSODA', rtol=3*eps, atol=eps)

# Plot the solution
plt.plot(sol_nl.y[0, :], sol_nl.y[1, :], label='Nonlinear orbit')

plt.plot(XL1[0], XL1[1], 'ro', label='L1')
# Dont stretch the plot
plt.axis('equal')
plt.legend()
plt.show()

# Compute the C value of the orbit
C_nl = Jacobi(Y0[0:2], Y0[2:4], mu)
print(C_nl)


In [ ]:
### PAC (Pseudo Arclength Continuation) cell

# We have DX, DF and F_X of the last correct iteration
# Find tau_tan as null(DF) and Y0 as the last correct initial state
X_k_old = Y0[0:4]
delta_s = 1e-3

tan_tau = splin.null_space(DF) # Tau is now a row vector
print(f"tan_tau: {tan_tau}")


Y_guess_old = Y_guess.reshape(3,1)
Y_guess_new = Y_guess_old + delta_s*tan_tau
print(f"Y_guess_old: {Y_guess_old}")
print(f"Y_guess_new: {Y_guess_new}")




In [ ]:

# Solve again the TPBVP
G_X = np.ones((3,1))
DG = np.zeros((3,3))


while np.linalg.norm(G_X) > 1e-12:
    # Shoot
    DX, DF, F_X = shooting(Y_guess_new, mu)

    # G_X = [F_X(2x1), tan_tau'*(Y_guess_new - Y_guess_old)-delta_s(1x1)]
    G_X[0:2] = F_X
    G_X[2] = tan_tau.T@(Y_guess_new - Y_guess_old) - delta_s
    if (np.linalg.norm(G_X) > 1e-12):
        # DG = [DF; tau.T]
        DG[0:2, :] = DF
        DG[2,:] = tan_tau.T
        # Update guess
        Y_guess_new = Y_guess_new - np.linalg.inv(DG)@G_X
    else:
        # We verified that |G_X| < 1e-12, so continue
        # to exit the loop
        continue


# For good measure, print Y_guess
print(f"Y_guess: {Y_guess_new}")
print(f"Accuracy: {F_X}")

In [ ]:
# Now again, take the found initial state and plot it
Y_guess_new = Y_guess_new.flatten()
Y0_1 = [Y_guess_new[0], 0, 0, Y_guess_new[1]]
Y0_1 = np.concatenate((Y0_1, PHI0.flatten()))
T = Y_guess_new[2]

tau_span = np.linspace(0, 1, 1000)
sol_nl_1 = solve_ivp(fdyn, [0, 1], Y0_1, args = (T, mu), t_eval=tau_span, method='LSODA', rtol=3*eps, atol=eps)

# Plot the solution
plt.plot(sol_nl.y[0, :], sol_nl.y[1, :],"r", label='Nonlinear orbit')
plt.plot(sol_nl_1.y[0, :], sol_nl_1.y[1, :], label='Nonlinear orbit 1')

plt.plot(XL1[0], XL1[1], 'ro', label='L1')
# Dont stretch the plot
plt.axis('equal')
plt.legend()
plt.show()

# Compute the C value of the orbit
C_nl = Jacobi(Y0_1[0:2], Y0_1[2:4], mu)
print(C_nl)




In [ ]:
xL1 = XL1.flatten()[0]
xL2 = XL2.flatten()[0]

X0 = sol_nl.y[:4, 0]
x0 = sol_nl.y[0, 0]

print(f"X0: {X0}")
print(f"x0: {x0}")
print(f"xL1: {xL1}")
print(f"xL2: {xL2}")

# Absolute distance from L1
dL1 = np.abs(x0 - xL1)
# Absolute distance from L2
dL2 = np.abs(x0 - xL2)

print(f"dL1: {dL1}")
print(f"dL2: {dL2}")

In [ ]:
# Search space
a = 0 
b = abs(1-mu - xL1)/2

# Initial guess is abs(x0 - xL1)
XG = np.array(X0)
Ydg = np.array([XG[0], XG[3], 2.69178148])
xg = abs(x0 - xL1)

Cg = Jacobi(X0[:2], X0[2:], mu)
print(f"Cg: {Cg}")

aux_bool = 0


In [ ]:
PIPPO, PLUTO = compute_orbit_Yd(Ydg, mu)

# Plot the solution
plt.plot(PIPPO[0, :], PIPPO[1, :], label='Nonlinear orbit 1')
plt.axis('equal')

In [ ]:
print(type(XG))

In [ ]:
while Cg < C_range[0] or Cg > C_range[1]:
    if Cg < C_range[0]:
        # Get closer to L1
        if (xg > 1-mu - xL1):
            b = 1-mu - xL1
        b = xg
    elif Cg > C_range[1]:
        # Get further from L1
        a = xg   
    xg = (a+b)/2
    # Update the initial state
    # We place the initial point to the right of the Lagrangian point
    XG = np.array([xL1 + xg, 0, 0, XG[3]])
    print("-----------------")
    print(f"XG: {XG}")
    # Shoot the orbit (and also update Ydg)
    Ydg, DF, F_X = shooting_loop(XG, Ydg[2], mu, num_iter=100)
    # Reassemble the state to compute the Jacobi constant
    Ydg = Ydg.flatten()
    XG = np.array([Ydg[0], 0, 0, Ydg[1]] )
    xg = abs(XG[0] - xL1)
    print(f"XG: {XG}")


    Cg = Jacobi(XG[:2], XG[2:], mu)
    print(f"Cg: {Cg}")
    print("-----------------")

# Print the final Cg

print(f"Final Cg: {Cg}")

    
#Cg: 3.1882793152572595
# XG: [ 0.91668008  0.          0.         -0.4049684 ]
# converget at XG: [ 0.87318809  0.          0.         -0.24964447]
# Final Cg: 3.1435628864200695


In [ ]:
# Plot XG
Ydg = Ydg.flatten()
Y0_2 = [Ydg[0], 0, 0, Ydg[1]]
Y0_2 = np.concatenate((Y0_2, PHI0.flatten()))
T = Ydg[2]

tau_span = np.linspace(0, 1, 1000)
sol_nl_2 = solve_ivp(fdyn, [0, 1], Y0_2, args = (T, mu), t_eval=tau_span, method='LSODA', rtol=3*eps, atol=eps)

# Plot the solution

plt.plot(sol_nl_2.y[0, :], sol_nl_2.y[1, :], label='Nonlinear orbit 2')

#### MAIN1.IPYNB